# Mean Daily Ice Surface Temperature

Download all the .nc files from below links with considering necessary filters of Date and Area

https://cvl.eo.esa.int/metsis/search?search_api_fulltext_op=and&fulltext=greenland+temperatur&start_date=&end_date=&is_parent=All&platform_ancillary_cloud_coverage=All&f%5B0%5D=dataset_level%3ALevel-1

https://nsidc.org/data/modgrnld/versions/1

chose NASA search tool

filter by time and area and (online available data only)

then use the "Simple mass downloader" Addon of Chrome to download all the necessary files to local drive

Give the local path in below cell

In [1]:
path = 'D://NC_files//**.nc'

In [2]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import netCDF4 as nc
from glob import glob


In [3]:
def find_date(ds):
    # This functions reads the data_date
    date = None
    for var in ds.variables.values():
        data_date = var.data_date
        assert date == None or data_date == date , 'Date Missmatch in data package for day {}'.format(data_date)
        date = data_date
    return date

In [4]:
def get_data(ds):
    #This function returns a Pandas dataframe from the daily average values of each feature
    df = pd.DataFrame()
    df ['Date'] = [find_date(ds)]
    df['Ice_Surface_Temperature_Mean'] = [np.mean(ds['Ice_Surface_Temperature_Mean'][0, :, :])]
    df['Ice_Surface_Temperature_Melt_NoMelt'] = [np.mean(ds['Ice_Surface_Temperature_Melt_NoMelt'][0, :, :])]
    df['Albedo'] = [np.mean(ds['Albedo'][0, :, :])]
    df['Water_Vapor_Near_Infrared_Mean'] = [np.mean(ds['Water_Vapor_Near_Infrared_Mean'][0, :, :])]
    df['NSwaths_and_Hour_Tracker'] = [np.mean(ds['NSwaths_and_Hour_Tracker'][0, :, :])]
    return df

    

In [5]:

files = glob(path)
print ('There are {} [.nc] files'.format(len(files)))

There are 1462 [.nc] files


In [6]:
#Generating the output table with taking the first file
output = get_data(nc.Dataset(files[0]))

for file in files[1:]:
    new_df = get_data(nc.Dataset(file))
    output = pd.concat([output, new_df], ignore_index=True,axis=0)
output.head()

    

,Date,Ice_Surface_Temperature_Mean,Ice_Surface_Temperature_Melt_NoMelt,Albedo,Water_Vapor_Near_Infrared_Mean,NSwaths_and_Hour_Tracker
0,2011-05-17,177.921143,19.981489,120.932588,0.528040,4.443885e+07
1,2011-05-18,177.252090,19.896570,121.026815,0.472696,4.549297e+07
2,2011-05-19,175.649979,20.297530,121.868638,0.494886,4.748086e+07
3,2011-05-20,176.409454,20.053332,123.472787,0.499137,3.413768e+07
4,2011-05-21,181.289764,18.993241,122.114159,0.473738,4.343364e+07


In [7]:
output.to_csv('Daily_data.csv')